# RSA Encryption

RSA was invented by NORAD in the early 1970's but was deemed top secret. A few years later in 1977, Ron **R**ivest, Adi **S**hamir and Leonard **A**dleman independantly invented the exact same process for encryption and called it RSA.

It shares many mathematical similarities with Diffie-Hellman encryption, but it's conceptually a little different. Instead of thinking about public and private keys, RSA can be thought of as sending an unlocked padlock. The receiver of the message has a key and a padlock. He sends his **unlocked** padlock to the sender who then locks his message with the padlock. Then when it's sent back to him he knows the key for decrpytion.

The problem RSA solves compared to Diffie-Hellman is that of key storage. When using Diffie-Hellman every key needs to be saved separately. This can be unwieldy for a large organizaion like a bank with many customers- the bank would need a different key for each customer. The solution to this is RSA!

# RSA Encryption Process

RSA encryption is a widely-used public key cryptosystem for secure data transmission. It involves mathematical operations based on prime numbers. Below is a detailed description of how RSA encryption works:

## Key Generation
1. **Select Two Prime Numbers**: Choose two large prime numbers, `p` and `q`.
2. **Compute the Modulus `n`**: Calculate `n = p * q`. The value of `n` is used as the modulus for both the public and private keys.
3. **Calculate the Totient (phi $φ$)**: Compute Euler's totient function, `φ(n) = (p-1) * (q-1)`.
4. **Choose the Public Exponent `e`**: Select an integer `e` such that `1 < e < φ(n)` and `e` is coprime with `φ(n)`. Often, `e = 65537`.
5. **Determine the Private Exponent `d`**: Calculate `d` as the modular multiplicative inverse of `e` modulo `φ(n)`.

## Encryption Process
1. **Public Key**: The public key consists of the modulus `n` and the public exponent `e`.
2. **Encrypting a Message**: Convert the message into an integer `m` (e.g., via ASCII). Then, compute the ciphertext `c` using $c ≡ m^e (mod n)$.

## Decryption Process
1. **Private Key**: The private key is made of the modulus `n` and the private exponent `d`.
2. **Decrypting a Message**: Decrypt the message by computing $m ≡ c^{d} (mod n)$ using the private key.

## Security
- RSA's security relies on the difficulty of factoring the product of two large prime numbers.
- The security improves with larger key sizes but at the cost of slower encryption and decryption.

## Practical Considerations
- RSA is often used to encrypt a symmetric key, which is then used for data encryption.
- Implementations must consider security aspects like padding schemes (e.g., OAEP) to prevent attacks.


# Helper Functions

In [ ]:
# Finds factors and determines if prime.

def factors(number):
    factors = set()
    for i in range(1, int(number**0.5)+1):
        if (number/i)%1 == 0:
            factors.add(i)
            factors.add(number // i)  # Add the corresponding divisor

            
    if len(factors) > 2:
        return {'factors': factors, 'is_prime': False}
    else:
        return {'factors': factors, 'is_prime': True}
    

In [ ]:
# Determines greatest common divisor of 2 numbers

def gcd(a, b):
    while b:
        a, b = b, a % b
    return a

In [ ]:
# returns array of generators of a prime number

def calculate_generator(prime):
    if factors(prime)['is_prime'] == False:
        return False

    generators = []
  
    for i in range(2, prime):
        results = []
        for j in range(2, prime):
            results.append(pow(i, j, prime))

        if len(set(results)) == len(results):
            generators.append(i)

    return generators


In [ ]:
# calulate phi - the totient

def calculate_phi(p, q):
    return (p - 1) * (q - 1)

# Selecting the public key

$e$ should be an integer that is not only greater than 1 but also less than $φ(n)$, where $n = pq$ (the product of two distinct prime numbers $p$ and $q$).

$e$ and $φ(n)$ should be coprime, meaning they should have no common factors other than 1. This ensures that $e$ has a multiplicative inverse $modulo φ(n)$.

In [ ]:
# Calculate if numbers are co-prime

def are_coprime(number1, number2):
    number1_factors = factors(number1)['factors']
    number2_factors = factors(number2)['factors']
    
    common_factors = number1_factors.intersection(number2_factors)
    
    # if the intersection of common factors is only 1 return True
    return common_factors == {1}


In [ ]:
# Calculate e - the public exponent

def exponent(totient):
    # Common practice is to start with 65537 (a prime number)
    e = 65537
    
    # Check if e is coprime with the totient
    # If not, decrement and check again
    while e > 2:
        if are_coprime(e, totient):
            return e
        e -= 2  # e must be an odd number

    raise ValueError("Could not find an appropriate 'e' value.")


# Calculate d - The private exponent

The private exponent `d` is the modular multiplicative inverse of `e modulo ϕ(n)`. 
This means `d` is the number that satisfies the equation 
$d × e ≡ 1 \ modϕ(n)$.


#def private_exponent():
    